<a href="https://colab.research.google.com/github/calisolo/Levels_image_captioning_NICE/blob/master/Levels_prepare_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%cd /content

!git clone https://github.com/calisolo/Levels_image_captioning_NICE.git
!git clone https://huggingface.co/datasets/calisolo/NICE_tsv/  #dataset


%cd ./Levels_image_captioning_NICE

/content
Cloning into 'Levels_image_captioning_NICE'...
remote: Enumerating objects: 689, done.
remote: Counting objects: 100% (159/159), done.
remote: Compressing objects: 100% (113/113), done.
remote: Total 689 (delta 77), reused 121 (delta 46), pack-reused 530
Receiving objects: 100% (689/689), 89.37 MiB | 27.74 MiB/s, done.
Resolving deltas: 100% (308/308), done.
Cloning into 'NICE_tsv'...
remote: Enumerating objects: 9, done.
remote: Counting objects: 100% (9/9), done.
remote: Compressing objects: 100% (8/8), done.
remote: Total 9 (delta 1), reused 0 (delta 0), pack-reused 0
Unpacking objects: 100% (9/9), 1.25 KiB | 1.25 MiB/s, done.
Filtering content: 100% (2/2), 890.00 MiB | 137.31 MiB/s, done.
/content/Levels_image_captioning_NICE


In [2]:
!pip install transformers==4.20.0
!pip install loguru

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 55.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 224.5/224.5 kB 26.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.6/6.6 MB 119.0 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.0/60.0 kB 6.2 MB/s eta 0:00:00


In [3]:
import pandas as pd 
val_set = pd.read_csv(f'data/nice-val-5k.csv')
pic_path = '../drive/MyDrive/NICE/cvpr-nice-test' ## you need to put test images to here! (in my case it was my google drive)

##image data, tsv format(img_id, '\t', img_content)(base64 encoding):

In [ ]:
from PIL import Image
from io import BytesIO
from tqdm import tqdm
import base64

result = []
for pic_id in  tqdm(val_set['public_id'], total = len(val_set)):
  img = Image.open(f'{pic_path}cvpr-nice-test/{pic_id}.jpg') # path to file
  img_buffer = BytesIO()
  img.save(img_buffer, format=img.format)
  byte_data = img_buffer.getvalue()
  base64_str = base64.b64encode(byte_data) # bytes
  base64_str = base64_str.decode("utf-8") # str

  resultStr = str(pic_id) + '\t' + base64_str
  result.append(resultStr)
  

100%|██████████| 21377/21377 [1:32:48<00:00,  3.84it/s]


In [ ]:
import csv
resultDF = pd.DataFrame(result)
resultDF.to_csv(f'./train_image(need to delete backslash).tsv',sep='\t', encoding = 'utf-8', index = False, header=None, quoting=csv.QUOTE_NONE,escapechar='\\' ) 
#A phenomenon in which double quotation marks are saved back and forth in str.. Need to re-upload after saving

## make cosine similarities map

In [4]:
#load model to get cosine similiarities from encoderOutput
from component.ofa.modeling_ofa import OFAModelForCaption, OFAModel
from component.ofa.tokenization_ofa import OFATokenizer 
from torchvision import transforms
import torch

model_name_or_path = 'calisolo/OFA_huge_image_captioning'
csv_path = '../NICE_tsv/'
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

model = OFAModelForCaption.from_pretrained(model_name_or_path).to(device)
tokenizer = OFATokenizer.from_pretrained('./vocab')

Downloading:   0%|          | 0.00/1.36k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.09G [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/torch/functional.py:504: UserWarning: torch.meshgrid: in an upcoming release, it will be required to pass the indexing argument. (Triggered internally at ../aten/src/ATen/native/TensorShape.cpp:3483.)
  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]


./vocab
<super: <class 'OFATokenizer'>, <OFATokenizer object>>


Calculate the cosine similarity of **valid set** values

In [ ]:
from PIL import Image
from tqdm import tqdm
from io import BytesIO
import base64

model.eval()

mean, std = [0.5, 0.5, 0.5], [0.5, 0.5, 0.5]
resolution = 256
patch_resize_transform = transforms.Compose([
        lambda image: image.convert("RGB"),
        transforms.Resize((resolution, resolution), interpolation=Image.BICUBIC),
        transforms.ToTensor(),
        transforms.Normalize(mean=mean, std=std)
    ])


txt = " what does the image describe?"
inputs = tokenizer([txt], return_tensors="pt").input_ids.to(device)
encoderDict = {}
image_file = f'{csv_path}train_image.tsv'

with open(image_file, 'r', encoding='utf8') as f:
  lines = f.readlines()

  with torch.no_grad():
    for line in tqdm(lines):
      pic_id, image_content = line.split('\t')
      img = Image.open(BytesIO(base64.urlsafe_b64decode(image_content)))
      patch_img = patch_resize_transform(img).unsqueeze(0).to(device)



      encodedOutput = model.encoder(input_ids = inputs, patch_images = patch_img)
      encoderDict[pic_id] = encodedOutput.last_hidden_state.cpu()


100%|██████████| 5000/5000 [05:48<00:00, 14.35it/s]


In [ ]:
def get_cosine_similarity(x1, x2):
    return (x1 * x2).sum() / ((x1**2).sum()**.5 * (x2**2).sum()**.5)

public_id_list = []
for i in encoderDict:
  public_id_list.append(i)

In [ ]:

import numpy as np
csMap = []

for index_i,i in tqdm(enumerate(public_id_list), total = len(public_id_list)):
  imsi = []
  for index_j,j in enumerate(public_id_list):
    if index_i == index_j:
      imsi.append(0)
      continue 
    cs = get_cosine_similarity(encoderDict[i], encoderDict[j]).item()
    imsi.append(cs)

  np_numbers = np.array(imsi)
  top_8_indices = np.argpartition(np_numbers, -8)[-8:]
  sorted_top_8_indices = top_8_indices[np.argsort(np_numbers[top_8_indices])[::-1]]

  num_8= {}
  for num in sorted_top_8_indices:
    num_8[val_set['public_id'][num]]= imsi[num]

  fin = [i, num_8]

  csMap.append(fin)

cspanda = pd.DataFrame(csMap)
cspanda.to_csv('./valid_encoderMap.csv',index= False)

  0%|          | 5/5000 [00:06<1:46:13,  1.28s/it]


KeyboardInterrupt: ignored

Calculate the cosine similarity value of **test set x valid set**

In [ ]:
from io import BytesIO
import base64
#train dataset도 encoding 뽑아보기

model.eval()
mean, std = [0.5, 0.5, 0.5], [0.5, 0.5, 0.5]
resolution = 256
patch_resize_transform = transforms.Compose([
        lambda image: image.convert("RGB"),
        transforms.Resize((resolution, resolution), interpolation=Image.BICUBIC),
        transforms.ToTensor(),
        transforms.Normalize(mean=mean, std=std)
    ])


txt = " what does the image describe?"
inputs = tokenizer([txt], return_tensors="pt").input_ids.to(device)
encoderDict_test = {}

image_file = f'{csv_path}test_image(need to delete backslash).tsv'

with open(image_file, 'r', encoding='utf8') as f:
  lines = f.readlines()

  with torch.no_grad():
    for line in tqdm(lines):
      pic_id, image_content = line.split('\t')
      
      pic_id = pic_id[:-1]##tsv에서 \\ 를 삭제안했을경우에..
      img = Image.open(BytesIO(base64.urlsafe_b64decode(image_content)))
      patch_img = patch_resize_transform(img).unsqueeze(0).to(device)



      encodedOutput = model.encoder(input_ids = inputs, patch_images = patch_img)
      encoderDict_test[pic_id] = encodedOutput.last_hidden_state.cpu()



  1%|          | 136/21377 [00:09<25:11, 14.05it/s]


KeyboardInterrupt: ignored

In [ ]:
public_id_list_test = []
for i in encoderDict_test:
  public_id_list_test.append(i)


In [ ]:

import numpy as np
csMap_test = []

for index_i,i in tqdm(enumerate(public_id_list_test), total = len(public_id_list_test)):
  imsi = []
  for index_j,j in enumerate(public_id_list):
    # if index_i >= index_j:
    #   imsi.append(0)
    #   continue 
    cs = get_cosine_similarity(encoderDict_test[i], encoderDict[j]).item()
    imsi.append(cs)

  np_numbers = np.array(imsi)
  top_8_indices = np.argpartition(np_numbers, -16)[-16:]
  sorted_top_8_indices = top_8_indices[np.argsort(np_numbers[top_8_indices])[::-1]]

  num_8= {}
  for num in sorted_top_8_indices:
    num_8[val_set['public_id'][num]]= imsi[num]

  fin = [i, num_8]

  csMap_test.append(fin)

  if len(csMap_test)% 2000==1:
    cspanda2 = pd.DataFrame(csMap_test)
    cspanda2.to_csv('./test_encoderMap.csv',index= False)


cspanda2 = pd.DataFrame(csMap_test)
cspanda2.to_csv('./test_encoderMap.csv',index= False)

##add location, shotstyle, category to encoder hint

In [ ]:
tag_added = pd.read_csv('./data/shotstyle_location worked.csv')
train_added = pd.read_csv('./data/encoder/test_cosine_parsed.csv')

In [ ]:
valid = pd.read_csv('./data/encoder/test_encoderMap.csv')
cosine = pd.read_csv('./data/encoder/test_fully_parsed_final.csv')

In [ ]:
train_added

,0,1,key1,key2,key3,key4,cos1,cos2,cos3,cos4,diff1,diff2,diff3,diff4
0,1587040490,"{1586724557: 0.2432771772146225, 1654577039: 0...",1586724557,1654577039,1590216599,1873340771,[cosHint lv1],[cosHint lv1],[cosHint lv1],[cosHint lv1],[diffHint lv1],[diffHint lv1],[diffHint lv1],[diffHint lv1]
1,1859182859,"{1859182757: 0.3702859580516815, 1859181131: 0...",1859182757,1859181131,1859182766,1587148232,[cosHint lv3],[cosHint lv3],[cosHint lv3],[cosHint lv3],[diffHint lv2],[diffHint lv2],[diffHint lv3],[diffHint lv1]
2,1766908167,"{1670342087: 0.35281312465667725, 1586672558: ...",1670342087,1586672558,1722076475,1586673437,[cosHint lv3],[cosHint lv3],[cosHint lv3],[cosHint lv2],[diffHint lv1],[diffHint lv1],[diffHint lv1],[diffHint lv1]
3,1587024551,"{1572535982: 0.30572494864463806, 1817411052: ...",1572535982,1817411052,1297774409,1576774418,[cosHint lv2],[cosHint lv2],[cosHint lv1],[cosHint lv1],[diffHint lv1],[diffHint lv1],[diffHint lv1],[diffHint lv1]
4,216584981,"{216586736: 0.3301239013671875, 216589541: 0.3...",216586736,216589541,1590350849,216582401,[cosHint lv3],[cosHint lv2],[cosHint lv2],[cosHint lv2],[diffHint lv2],[diffHint lv2],[diffHint lv1],[diffHint lv2]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21372,1570572905,"{1570573439: 0.4303014874458313, 1572527732: 0...",1570573439,1572527732,1570161023,1570572920,[cosHint lv4],[cosHint lv4],[cosHint lv3],[cosHint lv3],[diffHint lv2],[diffHint lv1],[diffHint lv1],[diffHint lv3]
21373,1578224348,"{1578935525: 0.35002365708351135, 1590067799: ...",1578935525,1590067799,1578215051,1590059978,[cosHint lv3],[cosHint lv3],[cosHint lv3],[cosHint lv3],[diffHint lv1],[diffHint lv1],[diffHint lv2],[diffHint lv1]
21374,1586691827,"{1586691842: 0.3243945837020874, 1586690789: 0...",1586691842,1586690789,1586690921,1586691857,[cosHint lv3],[cosHint lv2],[cosHint lv2],[cosHint lv1],[diffHint lv3],[diffHint lv2],[diffHint lv2],[diffHint lv3]
21375,1869041693,"{1869041636: 0.4087737202644348, 1843607069: 0...",1869041636,1843607069,1866001004,1586666987,[cosHint lv4],[cosHint lv3],[cosHint lv2],[cosHint lv1],[diffHint lv3],[diffHint lv1],[diffHint lv1],[diffHint lv1]


In [ ]:
id_list = []
cat_list = []
val_dict = {}
val_cat = {}
for index, i in enumerate(tag_added['public_id']):
  val_dict[i] = tag_added['location'][index]
  id_list.append(i)

for index, i in enumerate(tag_added['public_id']):
  val_cat[i] = tag_added['category'][index]
  cat_list.append(i)

In [ ]:
def find_closest_numbers(target, num_list, k):
    num_list.sort(key=lambda x: abs(x - target))
    if num_list[0] == target:
      return num_list[1:k+1]
    else:
      return num_list[:k]


In [ ]:
import math
shots = []
locations = []
for id in train_added['0']:
  closest_numbers = find_closest_numbers(id, id_list, 6)
  imsishot = []
  imsilocation = []
  for shot in closest_numbers:
    if  type(val_dict[shot])!= float:
      difference = abs(int(id) - int(shot))
      if difference < 100:
        diff = '[diffHint lv3]'+'['+ val_cat[shot]+ ']'
      elif difference < 10000:
        diff = '[diffHint lv2]'+'['+ val_cat[shot]+ ']'
      else:
        diff = '[diffHint lv1]'+'['+ val_cat[shot]+ ']'
      imsilocation.append(diff + val_dict[shot])
  locations.append(imsilocation)

In [ ]:
cosine.to_csv('./test_fully_parsed_final_2.csv', index=False)

In [ ]:
cosine

,0,1,key1,key2,key3,key4,cos1,cos2,cos3,cos4,diff1,diff2,diff3,diff4,shots,locations,cat1,cat2,cat3,cat4
0,1587040490,"{1586724557: 0.2432771772146225, 1654577039: 0...",1586724557,1654577039,1590216599,1873340771,[cosHint lv1],[cosHint lv1],[cosHint lv1],[cosHint lv1],[diffHint lv1],[diffHint lv1],[diffHint lv1],[diffHint lv1],"['Close up', 'Close up of']",[],[outdoors],[misc],[outdoors],[office]
1,1859182859,"{1859182757: 0.3702859580516815, 1859181131: 0...",1859182757,1859181131,1859182766,1587148232,[cosHint lv3],[cosHint lv3],[cosHint lv3],[cosHint lv3],[diffHint lv2],[diffHint lv2],[diffHint lv3],[diffHint lv1],"['Close up shot of', 'Low angle vertical shot ...",[],[outdoors],[outdoors],[outdoors],[outdoors]
2,1766908167,"{1670342087: 0.35281312465667725, 1586672558: ...",1670342087,1586672558,1722076475,1586673437,[cosHint lv3],[cosHint lv3],[cosHint lv3],[cosHint lv2],[diffHint lv1],[diffHint lv1],[diffHint lv1],[diffHint lv1],['Low angle view of'],[],[outdoors],[outdoors],[outdoors],[outdoors]
3,1587024551,"{1572535982: 0.30572494864463806, 1817411052: ...",1572535982,1817411052,1297774409,1576774418,[cosHint lv2],[cosHint lv2],[cosHint lv1],[cosHint lv1],[diffHint lv1],[diffHint lv1],[diffHint lv1],[diffHint lv1],"['Blurred view of', 'View of', 'View of', 'Vie...",[[diffHint lv2][outdoors]Cape Town South Africa],[outdoors],[outdoors],[outdoors],[outdoors]
4,216584981,"{216586736: 0.3301239013671875, 216589541: 0.3...",216586736,216589541,1590350849,216582401,[cosHint lv3],[cosHint lv2],[cosHint lv2],[cosHint lv2],[diffHint lv2],[diffHint lv2],[diffHint lv1],[diffHint lv2],['Close up of'],[[diffHint lv2][outdoors]Alcazar Seville Spain...,[outdoors],[outdoors],[outdoors],[outdoors]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21372,1570572905,"{1570573439: 0.4303014874458313, 1572527732: 0...",1570573439,1572527732,1570161023,1570572920,[cosHint lv4],[cosHint lv4],[cosHint lv3],[cosHint lv3],[diffHint lv2],[diffHint lv1],[diffHint lv1],[diffHint lv3],['Aerial view of'],[[diffHint lv3][outdoors]Cuxiu Muni Amazon Riv...,[outdoors],[outdoors],[outdoors],[outdoors]
21373,1578224348,"{1578935525: 0.35002365708351135, 1590067799: ...",1578935525,1590067799,1578215051,1590059978,[cosHint lv3],[cosHint lv3],[cosHint lv3],[cosHint lv3],[diffHint lv1],[diffHint lv1],[diffHint lv2],[diffHint lv1],[],[],[misc],[indoors],[misc],[indoors]
21374,1586691827,"{1586691842: 0.3243945837020874, 1586690789: 0...",1586691842,1586690789,1586690921,1586691857,[cosHint lv3],[cosHint lv2],[cosHint lv2],[cosHint lv1],[diffHint lv3],[diffHint lv2],[diffHint lv2],[diffHint lv3],[],[],[misc],[misc],[misc],[misc]
21375,1869041693,"{1869041636: 0.4087737202644348, 1843607069: 0...",1869041636,1843607069,1866001004,1586666987,[cosHint lv4],[cosHint lv3],[cosHint lv2],[cosHint lv1],[diffHint lv3],[diffHint lv1],[diffHint lv1],[diffHint lv1],"['Profile of', 'Front view portrait of', 'Elev...",[[diffHint lv3][outdoors]western cape in south...,[outdoors],[outdoors],[outdoors],[misc]


##caption data, jsonl format:
{"image_id": "1813180760", "text": ["A vertical shot of sunset on a beach"], "encoder_prefix": "[cosHint lv3][diffHint lv1]A landscape shot of sunset at horizon over ocean[cosHint lv3][diffHint lv1]Sun beach and ocean at Gerrans Bay Cornwall United Kingdom[cosHint lv3][diffHint lv1]Vertical shot of a beautiful sunset over the sea[cosHint lv3][diffHint lv1]Sunrise near Los Islotes Baja California Sur Mexico"}

form of a dictionary so that it can be called with a caption suitable for the public id when desired.

In [ ]:
!pip install jsonlines

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
val_set

,public_id,caption_gt,category
0,216582038,View of snowy mountain range,outdoors
1,1590061574,A young woman lying on a bed,misc
2,1868714447,A horizontal view of a scientist in lab suit e...,office
3,1813180760,A vertical shot of sunset on a beach,outdoors
4,1578946151,A woman relaxing in a deck chair,outdoors
...,...,...,...
4995,1587836972,Father and young children cooking in kitchen,indoors
4996,217367066,Mountains and forest in Canada,outdoors
4997,1571687297,View to Ronne Bornholm Island Denmark,outdoors
4998,1297775183,Young businessman in busy office looking at ca...,office


In [ ]:
val_dict = {}
for index, i in enumerate(val_set['public_id']):
  val_dict[i] = val_set['caption_gt'][index]

In [ ]:
pwd

'/content/Levels_image_captioning_NICE'

In [ ]:
valid = pd.read_csv('./data/encoder/valid_encoderMap.csv')  ## get cosine similarities map 
cosine = pd.read_csv('./data/encoder/train_fully_parsed_final_2.csv') 
valid

,0,1
0,216582038,"{216581957: 0.5075774192810059, 216578222: 0.4..."
1,1590061574,"{1590061697: 0.4025574028491974, 1590061787: 0..."
2,1868714447,"{1587990881: 0.3821038007736206, 1868714480: 0..."
3,1813180760,"{1811160932: 0.3904225826263428, 1859201003: 0..."
4,1578946151,"{1578947159: 0.36091023683547974, 1817411052: ..."
...,...,...
4995,1587836972,"{1576745393: 0.34038618206977844, 1813179776: ..."
4996,217367066,"{1572381332: 0.37447425723075867, 1570300802: ..."
4997,1571687297,"{1590338072: 0.4009000360965729, 1571335145: 0..."
4998,1297775183,"{1839580703: 0.38302579522132874, 1590061580: ..."


In [ ]:
cosine

,0,1,key1,key2,key3,key4,cos1,cos2,cos3,cos4,diff1,diff2,diff3,diff4,shots,locations,cat1,cat2,cat3,cat4
0,216582038,"{216581957: 0.5075774192810059, 216578222: 0.4...",216581957,216578222,216581945,1586673371,[cosHint lv4],[cosHint lv4],[cosHint lv4],[cosHint lv4],[diffHint lv3],[diffHint lv2],[diffHint lv3],[diffHint lv1],"['View of', 'View of', 'View of']",['[diffHint lv2][outdoors]Jarnac and the Chare...,[outdoors],[outdoors],[outdoors],[outdoors]
1,1590061574,"{1590061697: 0.4025574028491974, 1590061787: 0...",1590061697,1590061787,1576745306,1588012475,[cosHint lv4],[cosHint lv3],[cosHint lv2],[cosHint lv2],[diffHint lv2],[diffHint lv2],[diffHint lv1],[diffHint lv1],[],[],[misc],[misc],[outdoors],[misc]
2,1868714447,"{1587990881: 0.3821038007736206, 1868714480: 0...",1587990881,1868714480,1872077435,1868714486,[cosHint lv3],[cosHint lv3],[cosHint lv3],[cosHint lv3],[diffHint lv1],[diffHint lv3],[diffHint lv1],[diffHint lv3],"['A portrait of', 'A horizontal view of', 'A f...",[],[indoors],[office],[office],[office]
3,1813180760,"{1811160932: 0.3904225826263428, 1859201003: 0...",1811160932,1859201003,1572527741,216779531,[cosHint lv3],[cosHint lv3],[cosHint lv3],[cosHint lv3],[diffHint lv1],[diffHint lv1],[diffHint lv1],[diffHint lv1],"['Close up shot of', 'A portrait of']",[],[misc],[outdoors],[outdoors],[outdoors]
4,1578946151,"{1578947159: 0.36091023683547974, 1817411052: ...",1578947159,1817411052,1571580875,1578942776,[cosHint lv3],[cosHint lv3],[cosHint lv2],[cosHint lv2],[diffHint lv2],[diffHint lv1],[diffHint lv1],[diffHint lv2],['Portrait of'],[],[outdoors],[outdoors],[outdoors],[outdoors]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,1587836972,"{1576745393: 0.34038618206977844, 1813179776: ...",1576745393,1813179776,1665809777,1859336927,[cosHint lv3],[cosHint lv3],[cosHint lv2],[cosHint lv2],[diffHint lv1],[diffHint lv1],[diffHint lv1],[diffHint lv1],[],"[""[diffHint lv2][outdoors]Saint Peter's Basili...",[misc],[indoors],[food],[misc]
4996,217367066,"{1572381332: 0.37447425723075867, 1570300802: ...",1572381332,1570300802,1567877678,1567878608,[cosHint lv3],[cosHint lv3],[cosHint lv3],[cosHint lv3],[diffHint lv1],[diffHint lv1],[diffHint lv1],[diffHint lv1],[],['[diffHint lv3][outdoors]Lauterbrunnen Switze...,[outdoors],[outdoors],[outdoors],[outdoors]
4997,1571687297,"{1590338072: 0.4009000360965729, 1571335145: 0...",1590338072,1571335145,1571686673,1587842159,[cosHint lv4],[cosHint lv3],[cosHint lv3],[cosHint lv3],[diffHint lv1],[diffHint lv1],[diffHint lv2],[diffHint lv1],[],['[diffHint lv3][outdoors]Ravensburg Baden Wur...,[outdoors],[outdoors],[outdoors],[outdoors]
4998,1297775183,"{1839580703: 0.38302579522132874, 1590061580: ...",1839580703,1590061580,1839584162,1587139106,[cosHint lv3],[cosHint lv3],[cosHint lv3],[cosHint lv3],[diffHint lv1],[diffHint lv1],[diffHint lv1],[diffHint lv1],[],[],[office],[electronics],[office],[office]


In [ ]:
import jsonlines
import ast

# 예시 데이터
caption_result = []
imsi_result = {}
for index, public_id in enumerate(cosine['0']):
  processed_data = ''
  imsi_result['image_id'] = str(public_id)
  imsi_result['text'] = [val_dict[public_id]]

  processed_data = cosine['cos1'][index] + cosine['diff1'][index] + cosine['cat1'][index]+ val_dict[cosine['key1'][index]]
  processed_data += cosine['cos2'][index] + cosine['diff2'][index] + cosine['cat2'][index]+val_dict[cosine['key2'][index]]
  processed_data += cosine['cos3'][index] + cosine['diff3'][index] + cosine['cat3'][index]+val_dict[cosine['key3'][index]]
  processed_data += cosine['cos4'][index] + cosine['diff4'][index] + cosine['cat4'][index]+val_dict[cosine['key4'][index]]

  processed_data += '[shot_style]'

  
  shotlist = ast.literal_eval(cosine['shots'][index])
  locationlist = ast.literal_eval(cosine['locations'][index])
  if shotlist == []:
    processed_data += '[NULL]'
  else:
    for shot in shotlist:
      processed_data += shot + ' | '
  processed_data += '[Location]'
  if locationlist == []:
    processed_data += '[NULL]'
  else:
    for location in locationlist:
      processed_data += location + ' | '
  imsi_result['encoder_prefix'] = processed_data
  
  
  a = imsi_result.copy()
  caption_result.append(a)


In [ ]:

# 파일에 JSONNL 형식으로 데이터 저장
with jsonlines.open(f'data/train_caption_catAdded_2.jsonl', mode='w') as writer:
    for item in caption_result:
        writer.write(item)